In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model, Sequential
#from keras.layers import Dense, Embedding, LSTM, Input, ReLU
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.optimizers import Adam,Adamax,Nadam
from keras.layers import Dense, Embedding, LSTM, Input, ReLU, Conv1D, GlobalMaxPooling1D, Dropout, Activation
 
import re

In [ ]:
data = pd.read_csv('/kaggle/input/nnfl-lab-3-nlp/nlp_train.csv')
pd.set_option('display.max_colwidth',-1)
data.head()


In [ ]:
data.shape

In [ ]:
df=pd.read_csv("/kaggle/input/nnfl-lab-3-nlp/_nlp_test.csv",encoding="utf-8",index_col=0)
df = df.reset_index()
df = df.drop(['offensive_language'], axis=1)

In [ ]:
num_words = 5000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
print(data['tweet'][0])
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
print(X[0])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

maxlen = max([len(x) for x in X])
X = pad_sequences(X, maxlen=maxlen)

print(word_index)
print("Padded Sequences: ")
print(X)
print(X[0])


test = df.iloc[:,0]
test = list(test)
print(test)

X_t = tokenizer.texts_to_sequences(test)

X_t = pad_sequences(X_t, maxlen=maxlen)


print(X_t)
print(X_t[0])

In [ ]:
maxlen = max([len(x) for x in X])
maxlen

In [ ]:
embed_dim = 100
lstm_out = 128
batch_size = 64
model = Sequential()
model.add(Embedding(num_words,
                    embed_dim,
                    input_length = maxlen))
model.add(Dropout(0.20))
model.add(Conv1D(250,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('relu'))
model.summary()
optimizer = optimizers.SGD(lr=0.001 * 5, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
y = data['offensive_language']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
modelcheckpoint=ModelCheckpoint("weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size = batch_size, epochs = 30, callbacks=[earlystop,modelcheckpoint])

In [ ]:

# load weights into new model
model.load_weights("weights.13-0.33.hdf5")
print("Loaded model from disk")

In [ ]:
loss = model.evaluate(X_test, y_test, batch_size = batch_size, callbacks=[earlystop])
print(loss)

In [ ]:
df=pd.read_csv("/kaggle/input/nnfl-lab-3-nlp/_nlp_test.csv",encoding="utf-8",index_col=0)
df = df.reset_index()
df = df.drop(['offensive_language'], axis=1)


In [ ]:
df
df.shape

In [ ]:
df

In [ ]:



test = df.iloc[:,0]
test = list(test)
print(test)


In [ ]:
#num_words = 5000
#tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   #lower=True,split=' ')
#print(data['tweet'][0])
#tokenizer.fit_on_texts(data['tweet'].values)
X_t = tokenizer.texts_to_sequences(test)

#word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))


X_t = pad_sequences(X_t, maxlen=maxlen)


print(X_t)
print(X_t[0])

In [ ]:
y_pred = model.predict(X_t)

In [ ]:





print(y_pred)

In [ ]:
np.max(y_pred)

In [ ]:
y_pred[0]

In [ ]:
temp_1 = 3.0
for i in range(len(y_pred)):
    temp = y_pred[i]
    if temp > temp_1:
        temp = temp_1
    y_pred[i] = temp


        

    
        
    
        
        
    
    
        

In [ ]:
y_pred

In [ ]:
np.max(y_pred)

In [ ]:
df

In [ ]:


df["offensive_language"] = y_pred

In [ ]:
df

In [ ]:
df.to_csv("submission.csv")

In [ ]:




df

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):
  csv = df.to_csv(index=False)
  b64 = base64.b64encode(csv.encode())
  payload = b64.decode()
  html = '<a download="{filename}" href="data:text/csv;base64,{payload}"target="_blank">{title}</a>'
  html = html.format(payload=payload,title=title,filename=filename)
  return HTML(html)
create_download_link(df)
